In [ ]:
import cv2
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

# augmentation
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

tf.compat.v1.disable_v2_behavior()

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_generator = train_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = train_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32
)

baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(128, 128, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="sigmoid")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

plot_model(model, to_file='img/mobile-relu-sigmoid.png', show_shapes=True, show_layer_names=True)

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["acc"])

log_dir = "logs/fit/test-relu-sigmoid" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb = TensorBoard(histogram_freq=1, write_graph=True, write_grads=True, log_dir=log_dir)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

history = model.fit(
    x=train_generator,
    steps_per_epoch=len(train_generator) // 32,
    epochs=20,
    validation_data=val_generator,
    # validation_steps=len(val_generator) // 32,
    # validation_split=0.01
    callbacks=[reduce_lr, tb]
)

# model.evaluate(test_generator)

model.save('masknet-relu-sigmoid.h5')


In [ ]:
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# loading haarcascade_frontalface_default.xml
face_model = cv2.CascadeClassifier(
    '../input/haar-cascades-for-face-detection/haarcascade_frontalface_default.xml')
model = load_model('masknet-relu-sigmoid.h5')
mask_label = {0: 'MASK', 1: 'NO MASK'}
color_label = {0: (0, 255, 0), 1: (255, 0, 0)}
def detect(image_id):
    # Read in an image
    img = cv2.imread(f'../input/face-mask-detection/images/maksssksksss{image_id}.png')
    img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    # Detect faces
    faces = face_model.detectMultiScale(img)
    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image
    for i in range(len(faces)):
        (x, y, w, h) = faces[i]
        crop = new_img[y:y + h, x:x + w]
        crop = cv2.resize(crop, (128, 128))
        crop = np.reshape(crop, [1, 128, 128, 3]) / 255.0
        mask_result = model.predict(crop)
        color = color_label[mask_result.argmax()]
        cv2.putText(
            new_img,
            mask_label[mask_result.argmax()],
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            2
        )
        cv2.rectangle(new_img, (x, y), (x+w, y+h), color, 2)
    plt.figure(figsize=(10, 10))
    plt.imshow(new_img)

detect('331')
detect('455')